# In the Name of God
## Deep Learning HW3 - Problem 2
## Sayed Pouria fatemi 400206324


In [1]:
import tensorflow as tf

In [2]:
! git clone https://github.com/jimmyyhwu/resnet18-tf2.git

Cloning into 'resnet18-tf2'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 1), reused 6 (delta 1), pack-reused 0
Unpacking objects: 100% (6/6), 2.89 KiB | 2.89 MiB/s, done.


In [3]:
%cd /content/resnet18-tf2/
from resnet import resnet18

/content/resnet18-tf2


In [4]:
(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [5]:
training_data = tf.keras.applications.resnet50.preprocess_input(training_images)
validation_data = tf.keras.applications.resnet50.preprocess_input(validation_images)

##Part (a)

In [ ]:
base_model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights = "imagenet")
base_model.trainable = False

model = tf.keras.Sequential([
    tf.keras.layers.UpSampling2D((7, 7), interpolation='bilinear'),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

model.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(learning_rate = 5e-4) , metrics='accuracy')

history = model.fit(training_data,
                        training_labels,
                        epochs= 5,
                        validation_data=(validation_data, validation_labels),
                        batch_size= 100)

94765736/94765736 [==============================] - 4s 0us/step
Epoch 1/5
500/500 [==============================] - 179s 335ms/step - loss: 0.5159 - accuracy: 0.8281 - val_loss: 0.3432 - val_accuracy: 0.8833
Epoch 2/5
500/500 [==============================] - 186s 372ms/step - loss: 0.2949 - accuracy: 0.8997 - val_loss: 0.3124 - val_accuracy: 0.8936
Epoch 3/5
500/500 [==============================] - 186s 373ms/step - loss: 0.2566 - accuracy: 0.9124 - val_loss: 0.2858 - val_accuracy: 0.9020
Epoch 4/5
500/500 [==============================] - 186s 373ms/step - loss: 0.2333 - accuracy: 0.9193 - val_loss: 0.2739 - val_accuracy: 0.9081
Epoch 5/5
500/500 [==============================] - 186s 373ms/step - loss: 0.2168 - accuracy: 0.9262 - val_loss: 0.2735 - val_accuracy: 0.9071


<div dir="rtl"> <font size="4" face="vazirmatn">    
با توجه به این‌که فقط لایه 
Fully Connected
را آموزش می‌دهیم، با تنها ۵
epoch
آموزش به دقت ۹۰.۷۱٪ می‌رسیم که دقت قابل قبولی‌ست.
</font> </div> 

##Part (b)

<div dir="rtl"> <font size="4" face="vazirmatn">    
برای کد این قسمت از این سایت
https://keras.io/examples/vision/knowledge_distillation/
کمک گرفته شده است.
</font> </div> 

In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(self, alpha, temperature):
        super().compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 5e-4) , metrics='accuracy')
        self.student_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        self.distillation_loss_fn = tf.keras.losses.CategoricalCrossentropy()
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        teacher_predictions = self.teacher(data[0], training=False)

        with tf.GradientTape() as tape:
            student_predictions = self.student(data[0], training=True)
            student_loss = self.student_loss_fn(data[1], student_predictions)
            distillation_loss = self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1))
            loss = self.alpha * student_loss + (1 - self.alpha) * self.temperature**2 * distillation_loss

        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.compiled_metrics.update_state(data[1], student_predictions)
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss, "total_loss": loss})
        return results

    def test_step(self, data):
        y_prediction = self.student(data[0], training=False)
        student_loss = self.student_loss_fn(data[1], y_prediction)
        self.compiled_metrics.update_state(data[1], y_prediction)
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [6]:
def ResNet18(num_classes):
    inputs = tf.keras.layers.Input((32, 32, 3))
    y = tf.keras.layers.UpSampling2D((7, 7), interpolation='bilinear')(inputs)
    y = resnet18(y, num_classes=10)
    model = tf.keras.Model(inputs=inputs, outputs=y)
    return model

In [ ]:
temperatures = [1, 2, 3, 4]
alphas = [0.3, 0.5, 0.7, 0.9]
for temperature in temperatures:
  for alpha in alphas:
    print(f"temperature = {temperature}, alpha = {alpha}")
    student_model = ResNet18(num_classes=10)
    
    distiller = Distiller(student_model, model)
    distiller.compile(alpha, temperature)

    history_teacherStudent = distiller.fit(training_data,
                                training_labels,
                                epochs= 3,
                                validation_data=(validation_data, validation_labels),
                                batch_size= 100)

temperature = 1, alpha = 0.3
Epoch 1/3
500/500 [==============================] - 317s 627ms/step - accuracy: 0.5645 - student_loss: 1.2024 - total_loss: 1.2184 - val_accuracy: 0.6458 - val_student_loss: 1.0166
Epoch 2/3
500/500 [==============================] - 314s 629ms/step - accuracy: 0.7532 - student_loss: 0.7111 - total_loss: 0.7595 - val_accuracy: 0.7601 - val_student_loss: 0.7786
Epoch 3/3
500/500 [==============================] - 313s 627ms/step - accuracy: 0.8181 - student_loss: 0.5338 - total_loss: 0.6079 - val_accuracy: 0.7815 - val_student_loss: 0.7890
temperature = 1, alpha = 0.5
Epoch 1/3
500/500 [==============================] - 316s 626ms/step - accuracy: 0.5639 - student_loss: 1.2056 - total_loss: 1.2181 - val_accuracy: 0.5480 - val_student_loss: 1.3523
Epoch 2/3
500/500 [==============================] - 313s 627ms/step - accuracy: 0.7494 - student_loss: 0.7148 - total_loss: 0.7516 - val_accuracy: 0.7502 - val_student_loss: 0.7438
Epoch 3/3
500/500 [=============

<div dir="rtl"> <font size="4" face="vazirmatn">    
بهترین عملکرد را در حالت 
temperature = 3, alpha = 0.3
داریم.
</font> </div> 

In [ ]:
student_model = ResNet18(num_classes=10)

distiller = Distiller(student_model, model)
distiller.compile(alpha=0.3, temperature=3)

history_teacherStudent = distiller.fit(training_data,
                        training_labels,
                        epochs= 10,
                        validation_data=(validation_data, validation_labels),
                        batch_size= 100)

Epoch 1/10
500/500 [==============================] - 320s 634ms/step - accuracy: 0.5736 - student_loss: 1.3046 - total_loss: 10.6953 - val_accuracy: 0.6485 - val_student_loss: 1.2157
Epoch 2/10
500/500 [==============================] - 317s 633ms/step - accuracy: 0.7595 - student_loss: 0.7325 - total_loss: 9.1958 - val_accuracy: 0.7171 - val_student_loss: 1.0192
Epoch 3/10
500/500 [==============================] - 316s 633ms/step - accuracy: 0.8241 - student_loss: 0.5252 - total_loss: 8.7394 - val_accuracy: 0.8058 - val_student_loss: 0.5817
Epoch 4/10
500/500 [==============================] - 317s 633ms/step - accuracy: 0.8593 - student_loss: 0.4148 - total_loss: 8.5033 - val_accuracy: 0.8258 - val_student_loss: 0.6352
Epoch 5/10
500/500 [==============================] - 316s 632ms/step - accuracy: 0.8856 - student_loss: 0.3342 - total_loss: 8.3290 - val_accuracy: 0.8185 - val_student_loss: 0.4977
Epoch 6/10
500/500 [==============================] - 317s 634ms/step - accuracy: 0.

<div dir="rtl"> <font size="4" face="vazirmatn">    
دقت بدست آمده روی داده‌ی 
validation
قابل قبول است. همان‌طوری که در بالا مشاهده می‌کنید دقت نهایی ۸۶.۵۶٪ است.
</font> </div> 

##Part (c)

In [7]:
resnet18_new = ResNet18(num_classes=10)
resnet18_new.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(learning_rate = 5e-4) , metrics='accuracy')
history_resnet18_new = resnet18_new.fit(training_data,
                        training_labels,
                        epochs= 10,
                        validation_data=(validation_data, validation_labels),
                        batch_size= 100)

Epoch 1/10
500/500 [==============================] - 188s 353ms/step - loss: 1.2595 - accuracy: 0.5439 - val_loss: 1.0294 - val_accuracy: 0.6329
Epoch 2/10
500/500 [==============================] - 177s 354ms/step - loss: 0.7546 - accuracy: 0.7363 - val_loss: 0.7275 - val_accuracy: 0.7495
Epoch 3/10
500/500 [==============================] - 177s 353ms/step - loss: 0.5555 - accuracy: 0.8062 - val_loss: 0.7445 - val_accuracy: 0.7446
Epoch 4/10
500/500 [==============================] - 177s 355ms/step - loss: 0.4360 - accuracy: 0.8487 - val_loss: 0.5883 - val_accuracy: 0.8018
Epoch 5/10
500/500 [==============================] - 177s 354ms/step - loss: 0.3319 - accuracy: 0.8847 - val_loss: 0.5931 - val_accuracy: 0.8021
Epoch 6/10
500/500 [==============================] - 176s 353ms/step - loss: 0.2535 - accuracy: 0.9116 - val_loss: 0.5939 - val_accuracy: 0.8097
Epoch 7/10
500/500 [==============================] - 177s 353ms/step - loss: 0.1755 - accuracy: 0.9388 - val_loss: 0.6622 -

<div dir="rtl"> <font size="4" face="vazirmatn">    
همان‌طور که انتظار داشتیم دقت نهایی بدون وجود teacher
بدتر می‌شود. علت این امر این است که در وجود 
teacher
از عملکرد با دقت بالای یک شبکه‌ی دیگر کمک می‌گیریم و سعی در شبیه کردن بردارهای احتمال دو شبکه هستیم که در این‌جا به آن دسترسی نداریم.
</font> </div> 

##Part (d)

In [10]:
base_model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights = "imagenet")

resnet50_model = tf.keras.Sequential([
    tf.keras.layers.UpSampling2D((7, 7), interpolation='bilinear'),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

resnet50_model.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(learning_rate = 5e-4) , metrics='accuracy')

history_resnet50_model = resnet50_model.fit(training_data,
                        training_labels,
                        epochs= 5,
                        validation_data=(validation_data, validation_labels),
                        batch_size= 100)

Epoch 1/5
500/500 [==============================] - 541s 1s/step - loss: 0.4223 - accuracy: 0.8598 - val_loss: 0.4171 - val_accuracy: 0.8586
Epoch 2/5
500/500 [==============================] - 534s 1s/step - loss: 0.2075 - accuracy: 0.9282 - val_loss: 0.3302 - val_accuracy: 0.8901
Epoch 3/5
500/500 [==============================] - 534s 1s/step - loss: 0.1440 - accuracy: 0.9502 - val_loss: 0.3110 - val_accuracy: 0.9003
Epoch 4/5
500/500 [==============================] - 534s 1s/step - loss: 0.1048 - accuracy: 0.9632 - val_loss: 0.4110 - val_accuracy: 0.8806
Epoch 5/5
500/500 [==============================] - 534s 1s/step - loss: 0.0865 - accuracy: 0.9701 - val_loss: 0.3824 - val_accuracy: 0.8847


<div dir="rtl"> <font size="4" face="vazirmatn">    
همان‌طور که مشاهده می‌کنید دقت نسبت به قسمت (آ) کمی بدتر شده و زمان اجرا نیز افزایش یافته است که با توجه به این‌که کل پارمترهای مدل در حال آپدیت هستند طبیعی‌ست. همان‌طور که مشاهده می‌کنید دقت با حالت اول تفاوت جدی ندارد که نشان‌دهنده‌ی این است که شبکه
ResNet50
با وزن‌های آموزش دیده روی
ImageNet
ویژگی‌های خوبی برای دیتاست
CIFAR-10
استخراج می‌کند و نیاز به 
Fine-tuning
نداریم.

</font> </div> 